<a href="https://colab.research.google.com/github/Shivamfeb2/Complete-Python-3-Bootcamp/blob/master/DataIngestionToSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

#Connection details

hostname = "5vqyc.h.filess.io"
database = "olistproject_diagramdot"
port = "3307"
username = "olistproject_diagramdot"
password = "e128120a4b111a25a4fa5cce84afb2d5372c26df"

# CSV file path

csv_file_path = "olist_order_payments_dataset.csv"

# Table name where the data will be uploaded

table_name = "olist_order_payments"

try:
   # Step 1:- Establish a connection to mysql server
    connection = mysql.connector.connect(
        host=hostname, database=database,
        user=username, password=password,
        port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)

    # Step 2:- Create cursor to create SQL queries
        cursor = connection.cursor()

    # Step 3:- Drop table if it already exists (for clean insertion)
        cursor.execute(f"Drop table if exists {table_name};")
        record = cursor.fetchone()
        print(f"Table `{table_name}` dropped if existed.")

    # Step 4:- create a table structure to match the CSV file

    create_table_query = f"""
    CREATE TABLE {table_name} (
        order_id VARCHAR(255),
        payment_sequential INT,
        payment_type VARCHAR(255),
        payment_installments INT,
        payment_value DECIMAL(10, 2)
        )
    """
    cursor.execute(create_table_query)
    print(f"Table `{table_name}` created successfully.")

    # Step 5:- load the CSV data into pandas dataframe
    data = pd.read_csv(csv_file_path)
    print("CSV data loaded into a DataFrame.")

    # Step 6:- Insert data in batches of 500 records
    batch_size = 500 # Define the batch size
    total_records = len(data) # Get the total records in the DataFrame

    print(f"Starting data insertion into `{table_name}` in batches of {batch_size} records.")
    for start in range(0, total_records, batch_size):
      end = start + batch_size
      batch_data = data.iloc[start:end] # Get the current batch of records

    # Convert batch to list of tuples
    batch_records = [tuple(row) for row in batch_data.itertuples(index=False,name=None)]

    # Prepare the insert query

    insert_query = f"""INSERT INTO {table_name} (order_id, payment_sequential, payment_type,\
     payment_installments, payment_value) VALUES (%s, %s, %s, %s, %s)"""

    # Execute the insertiom query for the current batch

    cursor.executemany(insert_query, batch_records)
    connection.commit() # Commit after end of batch
    print(f"Inserted records {start + 1} to {min(end,total_records)} successfully")

    print(f"{len(batch_records)} records inserted into `{table_name}`.")

    print(f"All {total_records} records inserted into {table_name}")
except Error as e:
     #Step 7:- Handle the error if any
    print("Error while connecting to MySQL or inserting the data:", e)

finally:

    # Step 8:- Close the cursor and connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



Connected to MySQL Server version  8.0.36-28
Table `olist_order_payments` dropped if existed.
Table `olist_order_payments` created successfully.
CSV data loaded into a DataFrame.
Starting data insertion into `olist_order_payments` in batches of 500 records.
Inserted records 103501 to 103886 successfully
386 records inserted into `olist_order_payments`.
All 103886 records inserted into olist_order_payments
MySQL connection is closed


In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 13.2 MB/s eta 0:00:00


In [ ]:
# importing module
from pymongo import MongoClient

hostname = "b3o8q.h.filess.io"
database = "olistDataNoSQL_swimhollow"
port = "27018"
username = "olistDataNoSQL_swimhollow"
password = "f3b569ee7795fbad962c693b8a203a91db1500ec"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]


In [ ]:
# prompt: read the product_category csv and create a collection and upload it to above mongodb

import pandas as pd
from pymongo import MongoClient

# Assuming 'order_payments' DataFrame is already loaded as in the previous code.

# MongoDB connection details (replace with your actual credentials)
hostname = "b3o8q.h.filess.io"
database = "olistDataNoSQL_swimhollow"
port = "27018"
username = "olistDataNoSQL_swimhollow"
password = "f3b569ee7795fbad962c693b8a203a91db1500ec"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

try:
    # Connect to MongoDB
    client = MongoClient(uri)
    db = client[database]  # Access the database

    # Select the collection (or create it if it doesn't exist)
    collection = db["product_category"] # Replace 'product_category' with your desired collection name

    # Convert the DataFrame to a list of dictionaries
    data_dict = order_payments.to_dict("records")

    # Insert the data into the collection
    if data_dict:
        collection.insert_many(data_dict)
        print(f"Data successfully inserted into collection '{collection.name}'")
    else:
        print("DataFrame is empty. No data to insert.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the MongoDB connection
    if client:
        client.close()

In [ ]:
# prompt: read the product_category csv and create a collection and upload it to above mongoDB

import pandas as pd
from pymongo import MongoClient

# ... (Your existing code for MySQL connection and data loading) ...

# MongoDB connection details (replace with your actual credentials)
hostname = "b3o8q.h.filess.io"
database = "olistDataNoSQL_swimhollow"
port = "27018"
username = "olistDataNoSQL_swimhollow"
password = "f3b569ee7795fbad962c693b8a203a91db1500ec"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

try:
    # Connect to MongoDB
    client = MongoClient(uri)
    db = client[database]  # Access the database

    # Read the product_category CSV file into a pandas DataFrame
    product_category_df = pd.read_csv("product_category_name_translation.csv")  # Replace with the actual file path

    # Select the collection (or create it if it doesn't exist)
    collection = db["product_category"]

    # Convert the DataFrame to a list of dictionaries
    data_dict = product_category_df.to_dict("records")

    # Insert the data into the collection
    if data_dict:
        collection.insert_many(data_dict)
        print(f"Data successfully inserted into collection '{collection.name}'")
    else:
        print("DataFrame is empty. No data to insert.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the MongoDB connection
    if client:
        client.close()

Data successfully inserted into collection 'product_category'
